<a href="https://colab.research.google.com/github/AnandKumar56/Implementation_of_Chatbot_Using_NLP/blob/main/chatbot_week_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intent-based Chatbot


**Import necessary libraries**

In [5]:
import os
import nltk
import ssl
import streamlit as st
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

**Handling SSL Context for NLTK Downloads**

In [6]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**Load intents from a JSON file**

In [16]:
intents={
        "intents": [
                {
                        "tag": "greeting",
                        "patterns": [
                                "Hi",
                                "How are you",
                                "Is anyone there?",
                                "Hello",
                                "Good day",
                                "Whats up",
                                "how are ya",
                                "heyy",
                                "whatsup"
                        ],
                        "responses": [
                                "Hello!",
                                "Good to see you again!",
                                "Hi there, how can I help?"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "goodbye",
                        "patterns": [
                                "cya",
                                "see you",
                                "bye bye",
                                "See you later",
                                "Goodbye",
                                "I am Leaving",
                                "Bye",
                                "Have a Good day",
                                "talk to you later",
                                "tyyl",
                                "i got to go",
                                "gtg"
                        ],
                        "responses": [
                                "Sad to see you go :(",
                                "Talk to you later",
                                "Goodbye!",
                                "Come back soon"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "creator",
                        "patterns": [

                                "what is the name of your developers",
                                "what is the name of your creators",
                                "what is the name of the developers",
                                "what is the name of the creators",
                                "who created you",
                                "your developers",
                                "your creators",
                                "who are your developers",
                                "developers",
                                "you are made by",
                                "you are made by whom",
                                "who created you",
                                "who create you",
                                "creators",
                                "who made you",
                                "who designed you"
                        ],
                        "responses": [
                                "Ankush and Ahraar developed me in oct 2019, for their minor project",
                                "I was developed by Ahraar and Ankush",
                                "2 young boys developed me in CBPGEC college, Ankush and Ahraar"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "name",
                        "patterns": [
                                "name",
                                "your name",
                                "do you have a name",
                                "what are you called",
                                "what is your name",
                                "what should I call you",
                                "whats your name?",
                                "what are you",
                                "who are you",
                                "who is this",
                                "what am i chatting to",
                                "who am i taking to",
                                "what are you"
                        ],
                        "responses": [
                                "You can call me Rawkush.",
                                "I'm Rawkush",
                                "I'm Rawkush aka CBPGEC's Chatbot."
                        ],
                        "context_set": ""
                },
                {
                        "tag": "hours",
                        "patterns": [
                                "timing of college",
                                "what is college timing",
                                "working days",
                                "when are you guys open",
                                "what are your hours",
                                "hours of operation",
                                "when is the college open",
                                "college timing",
                                "what about college timing",
                                "is college open on saturday",
                                "tell something about college timing",
                                "what is the college  hours",
                                "when should i come to college",
                                "when should i attend college",
                                "what is my college time",
                                "college timing",
                                "timing college"
                        ],
                        "responses": [
                                "College is open 9am-5pm Monday-Friday!"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "number",
                        "patterns": [

                                "more info",
                                "contact info",
                                "how to contact college",
                                "college telephone number",
                                "college number",
                                "What is your contact no",
                                "Contact number?",
                                "how to call you",
                                "College phone no?",
                                "how can i contact you",
                                "Can i get your phone number",
                                "how can i call you",
                                "phone number",
                                "phone no",
                                "call"
                        ],
                        "responses": [
                                "You can contact at +91-011-25318157"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "course",
                        "patterns": [
                                "list of courses",
                                "list of courses offered",
                                "list of courses offered in cbpgec",
                                "what are the courses offered in your college?",
                                "courses?",
                                "courses offered",
                                "courses offered in cbpgec",
                                "courses you offer",
                                "branches?",
                                "courses available at GEC?",
                                "branches available at GEC?",
                                "what are the courses in GEC?",
                                "what are branches in GEC?",
                                "what are courses in GEC?",
                                "branches available in GEC?",
                                "can you tell me the courses available in GEC?",
                                "can you tell me the branches available in GEC?",
                                "Civil engineering?",
                                "civil",
                                "it",
                                "IT"
                        ],
                        "responses": [
                                "CBPGEC offers Information Technology, Civil Engineering and Mechanical Engineering."
                        ],
                        "context_set": ""
                },
                {
                        "tag": "fees",
                        "patterns": [
                                "information about fee",
                                "information on fee",
                                "tell me the fee",
                                "college fee",
                                "fee per semester",
                                "what is the fee of each semester",
                                "what is the fees of each year",
                                "what is fee",
                                "what is the fees",
                                "how much is the fees",
                                "fees for first year",
                                "fees",
                                "about the fees",
                                "tell me something about the fees"
                        ],
                        "responses": [
                                "For Fee detail visit <a target=\"_blank\" href=\"http://www.ipu.ac.in/Pubinfo2019/Admission2019mian/adm2019main.htm/\"> here</a>"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "location",
                        "patterns": [
                                "where is the college located",
                                "college is located at",
                                "where is college",
                                "where is college located",
                                "address of college",
                                "how to reach college",
                                "college location",
                                "college address",
                                "wheres the college",
                                "how can I reach college",
                                "whats is the college address",
                                "what is the address of college",
                                "address",
                                "location"
                        ],
                        "responses": [
                                "Ch. Brahm Prakash Government Engineering College, Lal Bagh Jafarpur Village, Delhi, 110073  <a target=\"_blank\" href=\"https://goo.gl/maps/cW8C4LDb81oSicRm6\"> here</a> "
                        ],
                        "context_set": ""
                },
                {
                        "tag": "hostel",
                        "patterns": [
                                "hostel facility",
                                "hostel servive",
                                "hostel location",
                                "hostel address",
                                "hostel facilities",
                                "hostel fees",
                                "Does college provide hostel",
                                "Is there any hostel",
                                "Where is hostel",
                                "do you have hostel",
                                "do you guys have hostel",
                                "hostel",
                                "hostel capacity",
                                "what is the hostel fee",
                                "how to get in hostel",
                                "what is the hostel address",
                                "how far is hostel from college",
                                "hostel college distance",
                                "where is the hostel",
                                "how big is the hostel",
                                "distance between college and hostel",
                                "distance between hostel and college"
                        ],
                        "responses": [
                                "Ch. Brahm Prakash Government Engineering College does  provide hostel facility"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "event",
                        "patterns": [
                                "events organised",
                                "list of events",
                                "list of events organised in college",
                                "list of events conducted in college",
                                "What events are conducted in college",
                                "Are there any event held at college",
                                "Events?",
                                "functions",
                                "what are the events",
                                "tell me about events",
                                "what about events"
                        ],
                        "responses": [
                                "proGECtion, Sports Week, Conference and various other event conducted in the college. For more information you can visit college Website <a target=\"_blank\" href=\"http://www.gecdelhi.ac.in/\"> here "
                        ],
                        "context_set": ""
                },
                {
                        "tag": "document",
                        "patterns": [
                                "document to bring",
                                "documents needed for admision",
                                "documents needed at the time of admission",
                                "documents needed during admission",
                                "documents required for admision",
                                "documents required at the time of admission",
                                "documents required during admission",
                                "What document are required for admission",
                                "Which document to bring for admission",
                                "documents",
                                "what documents do i need",
                                "what documents do I need for admission",
                                "documents needed"
                        ],
                        "responses": [
                                "To know more about document required visit <a target=\"_blank\" href=\"http://ipu.admissionhelp.com\"> here</a>"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "floors",
                        "patterns": [
                                "size of campus",
                                "building size",
                                "How many floors does college have",
                                "floors in college",
                                "floors in college",
                                "how tall is CBPGEC college building",
                                "floors"
                        ],
                        "responses": [
                                "Ch. Brahm Prakash Government Engineering College has total 2 floors "
                        ],
                        "context_set": ""
                },
                {
                        "tag": "syllabus",
                        "patterns": [
                                "Syllabus for IT",
                                "what is the Information Technology syllabus",
                                "syllabus",
                                "timetable",
                                "what is IT syllabus",
                                "syllabus",
                                "What is next lecture"
                        ],
                        "responses": [
                                "To know about  syllabus and timetable visit <a target=\"_blank\" href=\"http://www.gecdelhi.ac.in/tt_odd.html/\"> here</a>",
                                "To know about  syllabus and timetable visit <a target=\"_blank\" href=\"http://www.gecdelhi.ac.in/scheme_ITnew.html/\"> here</a>"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "library",
                        "patterns": [
                                "is there any library",
                                "library facility",
                                "library facilities",
                                "do you have library",
                                "does the college have library facility",
                                "college library",
                                "where can i get books",
                                "book facility",
                                "Where is library",
                                "Library",
                                "Tell me about library",
                                "how many libraries"
                        ],
                        "responses": [
                                "There is one library on second floor."
                        ],
                        "context_set": ""
                },
                {
                        "tag": "infrastructure",
                        "patterns": [
                                "how is college infrastructure",
                                "infrastructure",
                                "college infrastructure"
                        ],
                        "responses": [
                                "Ch. Brahm Prakash Government Engineering College has Excellent Infrastructure. Campus is clean. Good IT Labs With Good Speed of Internet connection"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "canteen",
                        "patterns": [
                                "food facilities",
                                "canteen facilities",
                                "canteen facility",
                                "is there any canteen",
                                "Is there a cafetaria in college",
                                "Does college have canteen",
                                "Where is canteen",
                                "where is cafetaria",
                                "canteen",
                                "Cafetaria"
                        ],
                        "responses": [
                                "CBPGEC has canteen with variety of food available"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "menu",
                        "patterns": [
                                "food menu",
                                "food in canteen",
                                "Whats there on menu",
                                "what is available in college canteen",
                                "what foods can we get in college canteen"
                        ],
                        "responses": [
                                "we serve Bread Pakoda, Chowmin, Thaali, Samosa, Dal Rice and many more on menu"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "placement",
                        "patterns": [
                                "What is college placement",
                                "Which companies visit in college",
                                "What is average package",
                                "companies visit",
                                "package",
                                "placement",
                                "recruitment",
                                "companies"
                        ],
                        "responses": [
                                "To know about placement visit <a target=\"_blank\" href=\"http://www.gecdelhi.ac.in/placement1%20first%20page.html/\">here</a>"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "ithod",
                        "patterns": [
                                "Who is IT HOD",
                                "Where is IT HOD",
                                "it hod",
                                "name of it hod"
                        ],
                        "responses": [
                                "Dr. Pankaj Lathar is IT HOD who is available on Ground floor"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "civilhod",
                        "patterns": [
                                "Who is Civil HOD",
                                "Where is civil HOD",
                                "civil hod",
                                "name of civil hod"
                        ],
                        "responses": [
                                "Dr. Athar Hussain is Civil HOD who is available on Ground floor"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "mechhod",
                        "patterns": [
                                "Who is Mechanical HOD",
                                "Where is  mechanical HOD",
                                "mechanical hod",
                                "name of mechanical hod"
                        ],
                        "responses": [
                                "Dr. Manju Nath K. is Mechanical HOD who is available on Ground floor"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "principal",
                        "patterns": [
                                "what is the name of principal",
                                "whatv is the principal name",
                                "principal name",
                                "Who is college principal",
                                "Where is principal's office",
                                "principal",
                                "name of principal"
                        ],
                        "responses": [
                                "Prof. K.C.Tiwari is college principal who is available on Ground floor"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "sem",
                        "patterns": [
                                "exam dates",
                                "exam schedule",
                                "When is semester exam",
                                "Semester exam timetable",
                                "sem",
                                "semester",
                                "exam",
                                "when is exam",
                                "exam timetable",
                                "exam dates",
                                "when is semester"
                        ],
                        "responses": [
                                "Here is the Academic Calendar  <a target=\"_blank\" href=\"http://www.gecdelhi.ac.in/pdf_files/academic%20calender%202018-19.pdf\">website</a>"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "admission",
                        "patterns": [
                                "what is the process of admission",
                                "what is the admission process",
                                "How to take admission in your college",
                                "What is the process for admission",
                                "admission",
                                "admission process"
                        ],
                        "responses": [
                                "Application can also be submitted online through the Unversity's  <a target=\"_blank\" href=\"www.ipu.ac.in\">website</a>"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "facilities",
                        "patterns": [
                                "What facilities college provide",
                                "College facility",
                                "What are college facilities",
                                "facilities",
                                "facilities provided"
                        ],
                        "responses": [
                                "CBPGEC provides fully AC Lab with internet connection, smart classroom, Auditorium, library,canteen"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "college intake",
                        "patterns": [
                                "max number of students",
                                "number of seats per branch",
                                "number of seats in each branch",
                                "maximum number of seats",
                                "maximum students intake",
                                "What is college intake",
                                "how many stundent are taken in each branch",
                                "seat allotment",
                                "seats"
                        ],
                        "responses": [
                                "For IT, Civil and Mechanical 60 per branch "
                        ],
                        "context_set": ""
                },
                {
                        "tag": "uniform",
                        "patterns": [
                                "college dress code",
                                "college dresscode",
                                "what is the uniform",
                                "can we wear casuals",
                                "Does college have an uniform",
                                "Is there any uniform",
                                "uniform",
                                "what about uniform",
                                "do we have to wear uniform"
                        ],
                        "responses": [
                                "CBPGEC does not have uniform"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "committee",
                        "patterns": [
                                "what are the different committe in college",
                                "different committee in college",
                                "Are there any committee in college",
                                "Give me committee details",
                                "committee",
                                "how many committee are there in college"
                        ],
                        "responses": [
                                "There are various committe in college you can see at  <a target=\"_blank\" href=\"http://www.gecdelhi.ac.in/grievance%20redressal%20sysytem.html\">here</a> "
                        ],
                        "context_set": ""
                },
                {
                        "tag": "random",
                        "patterns": [

                                "I love you",
                                "Will you marry me",
                                "Do you love me"
                        ],
                        "responses": [
                                "I am not program for this please ask appropriate query"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "swear",
                        "patterns": [
                                "fuck",
                                "bitch",
                                "shut up",
                                "hell",
                                "stupid",
                                "idiot",
                                "dumb ass",
                                "asshole",
                                "fucker"
                        ],
                        "responses": [
                                "please use appropriate language",
                                "Maintaining decency would be appreciated"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "vacation",
                        "patterns": [
                                "holidays",
                                "when will semester starts",
                                "when will semester end",
                                "when is the holidays",
                                "list of holidays",
                                "about vacations",
                                "about holidays",
                                "When is vacation",
                                "When is holidays",
                                "how long will be the vacation"
                        ],
                        "responses": [
                                "Please refer academic calendar <a target=\"_blank\" href=\"http://www.gecdelhi.ac.in/pdf_files/academic%20calender%202018-19.pdf\">here</a>"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "salutaion",
                        "patterns": [
                                "okk",
                                "okie",
                                "nice work",
                                "well done",
                                "good job",
                                "thanks for the help",
                                "Thank You",
                                "its ok",
                                "Thanks",
                                "k",
                                "ok",
                                "okay"
                        ],
                        "responses": [
                                "I am glad I helped you",
                                "welcome, anything else i can assist you with?"
                        ],
                        "context_set": ""
                },
                {
                        "tag": "task",
                        "patterns": [
                                "what can you do",
                                "what are the thing you can do",
                                "things you can do",
                                "what can u do for me",
                                "how u can help me",
                                "why i should use you"
                        ],
                        "responses": [
                                "I can answer to low-intermediate questions regarding college",
                                "You can ask me questions regarding college, and i will try to answer them"
                        ],
                        "context_set": ""
                },
                {
                        "tag":"ragging",
                        "patterns":[
                                "ragging",
                                "is ragging practice active in college",
                                "does college have any antiragging facility",
                                "is there any ragging cases",
                                "is ragging done here",
                                "ragging juniors",
                                "ragging history",
                                "ragging incidents"

                        ],
                        "responses": [
                                "We are Proud to tell you that our college provides ragging free environment, and we have strict rules against ragging"
                        ]

                },
                {
                        "tag":"hod",
                        "patterns":[
                                "hod",
                                "hod name",
                                "who is the hod"
                        ],
                        "responses":[
                                "HODs differ for each branch, please be more specific like: (HOD it)"
                        ]
                },
                {
                        "tag":"transport",
                        "patterns":[
                                "dtc bus route",
                                "mode of transportation",
                                "how to get to college",
                                "buses for college",
                                "college buses",
                                "bus routes",
                                "list of buses for travelling",
                                "does 835 goes to college",
                                "does 835 goes to rawta mor"

                        ],
                        "responses":[
                                "827,821,878: Jaffarpur Hospital,  \n 835,836,848: Rawta More"
                        ]
                },
                {
                        "tag": "metro",
                        "patterns":[
                                "nearest metro station",
                                "what is the nearest metro station",
                                "which metro station is near",
                                "closest metro station",
                                "which metro station is closest",
                                "nearby metro stations",
                                "metro",
                                "metro station nearby"
                        ],
                        "responses":[
                                "Najafgarh metro station"
                        ]
                }

        ]
}

In [7]:
intents_file = ""
with open(intents_file, 'r') as file:
    intents = json.load(file)

**Prepare data for training and Vectorize the text data**

In [9]:
# Create the vectorizer and classifier
vectorizer = TfidfVectorizer()
clf = LogisticRegression(random_state=0, max_iter=10000)

# Preprocess the data
tags = []
patterns = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)

# training the model
x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x, y)

LogisticRegression(max_iter=10000, random_state=0)

**Define the chatbot function**

In [10]:
def chatbot(input_text):
    try:
        input_text = input_text.lower().strip()
        input_vector = vectorizer.transform([input_text])
        tag = clf.predict(input_vector)[0]
        for intent in intents['intents']:
            if intent['tag'] == tag:
                return random.choice(intent['responses'])
    except Exception:
        return "I'm sorry, I didn't understand that. Can you try again?"

In [11]:
if __name__ == "__main__":
    print("Chatbot is ready! Type 'quit' to exit.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            print("Chatbot: Goodbye!")
            break
        response = chatbot(user_input)
        print(f"Chatbot: {response}")

Chatbot is ready! Type 'quit' to exit.
You: Hi
Chatbot: Hello!
You: Whatsapp
Chatbot: CBPGEC offers Information Technology, Civil Engineering and Mechanical Engineering.
You: What's the weather like?
Chatbot: For Fee detail visit <a target="_blank" href="http://www.ipu.ac.in/Pubinfo2019/Admission2019mian/adm2019main.htm/"> here</a>
You: Bye
Chatbot: Sad to see you go :(
You: quit
Chatbot: Goodbye!



**Thank You!!!**